In [17]:
library(forcats)
library(readxl)
library(ggplot2)
library(gridExtra)
library(dplyr)
library(leaps)
library(pROC)
library(stats)
library(bestglm)
library(glmnet)
library(ROCR)
library(FactoMineR) 
library(factoextra)
library(mclust)
library(labelled)
library(tidyverse)
library(randomForest)
library(caret)

In [18]:
insa <- read_excel("./survey.xlsx")
insa <- as.data.frame(insa)
insa <- insa[,2:53]

str(insa)

New names:
• `Tiers_temps` -> `Tiers_temps...9`
• `Presence_CM` -> `Presence_CM...16`
• `Presence_TD` -> `Presence_TD...17`
• `Presence_TP` -> `Presence_TP...18`
• `Tiers_temps` -> `Tiers_temps...90`
• `Presence_CM` -> `Presence_CM...133`
• `Presence_TD` -> `Presence_TD...134`
• `Presence_TP` -> `Presence_TP...135`
• `` -> `...415`
• `` -> `...416`
• `` -> `...417`
• `` -> `...418`
• `` -> `...419`


'data.frame':	1185 obs. of  52 variables:
 $ trajet               : chr  "15-30min" "5-15min" "0-5min" "5-15min" ...
 $ logement             : chr  "seul" "seul" "seul" "seul" ...
 $ surf_logement        : chr  "+ 30m2" "+ 30m2" "10-20m2" "+ 30m2" ...
 $ sent_logement        : chr  "tout à fait" "tout à fait" "plutôt oui" "tout à fait" ...
 $ act_pro              : chr  "aucune" "aucune" "aucune" "aucune" ...
 $ Bourse               : chr  "echelon 1-3" "non" "non" "non" ...
 $ handicap             : chr  "non" "non" "non" "non" ...
 $ Tiers_temps...9      : chr  "Non" "Non" "Non" "Non" ...
 $ Stress               : chr  "souvent" "souvent" "souvent" "toujours" ...
 $ travail_sem          : chr  "0-1h" "+4h" "1-2h" "1-2h" ...
 $ travail_wek          : chr  "2-4h" "2-4h" "4-6h" "6-8h" ...
 $ Travail_groupe       : chr  "jamais" "jamais" "parfois" "parfois" ...
 $ Travail_ext          : chr  "souvent" "parfois" "parfois" "parfois" ...
 $ Travail_dom          : chr  "parfois" "souvent" "s

In [21]:
# Fonction pour convertir les colonnes en facteur avec des niveaux triés
convert_to_factor <- function(data, column, levels_order) {
    data[[column]] <- factor(data[[column]], levels = levels_order, ordered = TRUE)
}

# Définir l'ordre des niveaux pour des variables spécifiques
levels_list <- list(
    trajet = c("0-5min", "5-15min", "15-30min", "30-45min", "45-60min", ">60min"),
    logement = c("seul", "colocation", "famille", "couple"),
    surf_logement = c("-10m2", "10-20m2", "20-30m2", "+ 30m2"),
    sent_logement = c("pas du tout", "plutôt non", "plutôt oui", "tout à fait"),
    act_pro = c("aucune", "CDD", "CDI", "indépendant"),
    Bourse = c("non", "echelon 0", "echelon 1-3", "echelon 4-6"),
    handicap = c("non", "oui"),
    Tiers_temps = c("Non", "Oui"),
    Stress = c("jamais", "rarement", "parfois", "souvent", "toujours"),
    travail_sem = c("0-1h", "1-2h", "2-3h", "3-4h", "4-5h", "5-6h", "6h et plus"),
    travail_wek = c("0-1h", "1-2h", "2-3h", "3-4h", "4-5h", "5-6h", "6h et plus"),
    Travail_groupe = c("jamais", "rarement", "parfois", "souvent", "toujours"),
    Travail_ext = c("jamais", "rarement", "parfois", "souvent", "toujours"),
    Travail_dom = c("jamais", "rarement", "parfois", "souvent", "toujours"),
    Presence_CM = c("Jamais", "Rarement", "Parfois", "Souvent", "Toujours"),
    Presence_TD = c("Jamais", "Rarement", "Parfois", "Souvent", "Toujours"),
    Presence_TP = c("Jamais", "Rarement", "Parfois", "Souvent", "Toujours"),
    Notes_cours = c("Jamais", "Rarement", "Parfois", "Souvent", "Toujours"),
    Concentration_CM = c("Pas du tout concentré.e", "Très peu concentré.e", "Moyennement concentré.e", "Assez bien concentré.e", "Totalement concentré.e"),
    Concentration_TD = c("Pas du tout concentré.e", "Très peu concentré.e", "Moyennement concentré.e", "Assez bien concentré.e", "Totalement concentré.e"),
    Concentration_TP = c("Pas du tout concentré.e", "Très peu concentré.e", "Moyennement concentré.e", "Assez bien concentré.e", "Totalement concentré.e"),
    Prep_CM = c("Jamais", "Rarement", "Parfois", "Souvent", "Toujours"),
    Prep_TD = c("Jamais", "Rarement", "Parfois", "Souvent", "Toujours"),
    Prep_TP = c("Jamais", "Rarement", "Parfois", "Souvent", "Toujours"),
    revisions = c("1 jour avant", "1 semaine avant", "2 semaines avant", "1 mois avant", "plus d'un mois avant"),
    efficacite = c("pas du tout", "plutôt non", "moyennement", "plutôt oui", "très"),
    travail_cond = c("très mauvaises", "mauvaises", "moyennes", "plutôt bonnes", "très bonnes"),
    ressources_sup = c("jamais", "rarement", "parfois", "souvent", "très souvent"),
    Amis = c("jamais", "rarement", "parfois", "souvent", "très souvent"),
    Famille = c("jamais", "rarement", "parfois", "souvent", "très souvent"),
    isol = c("jamais", "rarement", "parfois", "souvent", "toujours"),
    associatif = c("non", "0-1h", "1-5h", "5-10h", "10-15h", "15-20h", "20h et plus"),
    associatif_bien_etre = c("pas d'asso", "pas du tout", "plutôt non", "moyennement", "plutôt oui", "tout à fait"),
    associatif_fatigue = c("pas d'asso", "pas du tout", "plutôt non", "moyennement", "plutôt oui", "tout à fait"),
    sport = c("non", "0-1h", "1-3h", "3-6h", "6-9h", "plus de 9h"),
    Frequence_spectacle = c("jamais", "rarement", "1/mois", "2-3/mois", "1/semaine", "plusieurs fois/semaine"),
    Frequence_bar_boite = c("jamais", "rarement", "1/mois", "2-3/mois", "1/semaine", "plusieurs fois/semaine"),
    detente = c("jamais", "rarement", "parfois", "souvent", "très souvent"),
    ecrans = c("moins de 1h", "1-2h", "2-4h", "4-6h", "6-8h", "8-10h", "plus de 10h"),
    sommeil = c("très insatisfaisante", "plutôt insatisfaisante", "plutôt satisfaisante", "très satisfaisante"),
    alimentation = c("pas du tout", "plutôt non", "plutôt oui", "tout à fait"),
    Motivation = c("pas du tout", "plutôt non", "plutôt oui", "tout à fait"),
    Sensation_reussite = c("pas du tout", "plutôt non", "plutôt oui", "tout à fait"),
    Adequation_projet_pro = c("pas du tout", "plutôt non", "plutôt oui", "tout à fait"),
    Promo = c("1A", "2A", "3A", "4A", "5A"),
    Spe_PO = c("GB", "GMM", "GPE", "GIT", "GE"),
    Cohorte = c("classique", "alternant", "apprenti"),
    Bac_origine = c("Fr", "Etr", "Autre"),
    Resultat_1A = c("0-10%", "10-25%", "25-50%", "50-75%", "75-100%", "pas de classement"),
    Resultat_preced = c("0-10%", "10-25%", "25-50%", "50-75%", "75-100%"),
    Redoublement = c("Non", "Oui"),
    Rattrapages = c("aucun", "1-2", "3-5", "plus de 5")
)

# Appliquer la fonction de conversion à toutes les colonnes 2 à 53
for (col in colnames(insa)) {
    if (col %in% names(levels_list)) {
        convert_to_factor(insa, col, levels_list[[col]])
    }
}

# Revérifiez la structure des données après la conversion
str(insa)


'data.frame':	1185 obs. of  52 variables:
 $ trajet               : chr  "15-30min" "5-15min" "0-5min" "5-15min" ...
 $ logement             : chr  "seul" "seul" "seul" "seul" ...
 $ surf_logement        : chr  "+ 30m2" "+ 30m2" "10-20m2" "+ 30m2" ...
 $ sent_logement        : chr  "tout à fait" "tout à fait" "plutôt oui" "tout à fait" ...
 $ act_pro              : chr  "aucune" "aucune" "aucune" "aucune" ...
 $ Bourse               : chr  "echelon 1-3" "non" "non" "non" ...
 $ handicap             : chr  "non" "non" "non" "non" ...
 $ Tiers_temps...9      : chr  "Non" "Non" "Non" "Non" ...
 $ Stress               : chr  "souvent" "souvent" "souvent" "toujours" ...
 $ travail_sem          : chr  "0-1h" "+4h" "1-2h" "1-2h" ...
 $ travail_wek          : chr  "2-4h" "2-4h" "4-6h" "6-8h" ...
 $ Travail_groupe       : chr  "jamais" "jamais" "parfois" "parfois" ...
 $ Travail_ext          : chr  "souvent" "parfois" "parfois" "parfois" ...
 $ Travail_dom          : chr  "parfois" "souvent" "s